In [1]:
import numpy as np
import pandas as pd
from numpy.random import randint
import re
import collections as col
from IPython.core.display import display,HTML
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot, download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
import itertools as itr
from statsmodels.stats import weightstats
from datetime import datetime,timedelta

In [2]:
df=pd.read_csv('main_task.csv')    # загрузим
df.columns=['id','city','cuisine','ranking','rating','price_range','rev_nums','reviews','url','id_ta'] # переименуем
df=df.drop_duplicates(subset=['id_ta']).reset_index().drop(['index'],axis=1)       # eсли имеются дупликаты в датасете по id трипадвизора -дропнем эти дупликаты
df=df.drop(columns=['id','id_ta','url','reviews'],axis=1)   # также дропну слобцы с которыми не буду работать при подготовке модели
display(df.info())                   # действительно было  20 дупликатов с сете
df   #общий вид

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         39980 non-null  object 
 1   cuisine      30701 non-null  object 
 2   ranking      39980 non-null  float64
 3   rating       39980 non-null  float64
 4   price_range  26101 non-null  object 
 5   rev_nums     37437 non-null  float64
dtypes: float64(3), object(3)
memory usage: 1.8+ MB


None

,city,cuisine,ranking,rating,price_range,rev_nums
0,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0
1,Stockholm,NaN,1537.0,4.0,NaN,10.0
2,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0
3,Berlin,NaN,3458.0,5.0,NaN,3.0
4,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0
...,...,...,...,...,...,...
39975,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0
39976,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0
39977,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0
39978,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0


# Начнем заполнять пропуски и преобразовывать столбцы

In [3]:
df['cuisine']=df['cuisine'].apply(lambda x : x[1:-1].replace("'","") if pd.notna(x) else None)   # преобразуем кухни в списки строковых величин
df['cuisine']=df['cuisine'].apply(lambda x : x.replace(" ","") if pd.notna(x) else None)   # также уберем пробелы после запятых
prices=list(df['price_range'].value_counts().index)                                         # в столбце price_range 3 значение диапазонов цен               
df['price_range']=df['price_range'].apply(lambda x : 'low' if x==prices[1] else 'mid' if x==prices[2] else 'high' if x==prices[0] else None) # преобразуем их в high,low,mid

In [4]:
grouped=df.groupby(['city','price_range'])['price_range'].count()   # сгруппируем и посчитаем сколько в каких городах ресторанов каких ценовых категорий
grouped

city       price_range
Amsterdam  high           644
           low            163
           mid             31
Athens     high           275
           low            137
                         ... 
Warsaw     low            111
           mid             28
Zurich     high           293
           low             47
           mid             34
Name: price_range, Length: 93, dtype: int64

In [5]:
"""заполним вторым уровнем мультииндекса наиболее часто встречающимся в городе,содержащим ценовой диапазон если пропуск ,иначе оставим значение"""
df['price_range']=pd.Series(df.index).apply(lambda x : grouped.loc[df['city'].loc[x],:].idxmax()[1] if pd.isna(df['price_range'].loc[x]) else df['price_range'].loc[x])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39980 entries, 0 to 39979
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         39980 non-null  object 
 1   cuisine      30701 non-null  object 
 2   ranking      39980 non-null  float64
 3   rating       39980 non-null  float64
 4   price_range  39980 non-null  object 
 5   rev_nums     37437 non-null  float64
dtypes: float64(3), object(3)
memory usage: 1.8+ MB


# посчитаем среднее количество отзывов в различных городах и определенных ценовых группах

In [6]:
grouped2=df.groupby(['city','price_range'])['rev_nums'].mean()
grouped2

city       price_range
Amsterdam  high           139.297998
           low            125.070513
           mid            455.935484
Athens     high           105.876993
           low             57.201493
                             ...    
Warsaw     low             54.822917
           mid            220.857143
Zurich     high            71.014151
           low             41.851064
           mid            208.529412
Name: rev_nums, Length: 93, dtype: float64

In [7]:
"""заполним средним значением по городу и ценовой категории если пропуск , иначе оставим значение"""
df['rev_nums']=pd.Series(df.index).apply(lambda x : grouped2.loc[df['city'].loc[x],df['price_range'].loc[x]] if pd.isna(df['rev_nums'].loc[x]) else df['rev_nums'].loc[x] )

In [8]:
df['cuisine']=df['cuisine'].apply(lambda x :x.split(',') if pd.notna(x) else None)
cuisine=df['cuisine'].dropna(axis=0)
df['cuisine']=df['cuisine'].apply(lambda x : cuisine.iloc[randint(0,len(cuisine))] if x==None else x)
#df['cuisine_len']=df['cuisine'].apply(lambda x : len(x))
df

,city,cuisine,ranking,rating,price_range,rev_nums
0,Paris,"[European, French, International]",5570.0,3.5,high,194.0
1,Stockholm,"[FastFood, Mediterranean, MiddleEastern, Israe...",1537.0,4.0,high,10.0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,mid,688.0
3,Berlin,"[Belgian, European]",3458.0,5.0,high,3.0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,high,84.0
...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,high,79.0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,high,542.0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,high,4.0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,high,70.0


In [9]:
cuisines=[]
for el in cuisine:
    cuisines.extend(el)

In [10]:
new_columns=list(pd.Series(cuisines).value_counts().index)
temp_df=pd.DataFrame(dict.fromkeys(new_columns,[0 for i in range(0,len(df))]))
df=pd.concat([df,temp_df],axis=1)
df

,city,cuisine,ranking,rating,price_range,rev_nums,VegetarianFriendly,European,Mediterranean,Italian,...,Ecuadorean,Welsh,Polynesian,Azerbaijani,Fujian,Yunnan,Burmese,Salvadoran,Latvian,Xinjiang
0,Paris,"[European, French, International]",5570.0,3.5,high,194.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Stockholm,"[FastFood, Mediterranean, MiddleEastern, Israe...",1537.0,4.0,high,10.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,mid,688.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin,"[Belgian, European]",3458.0,5.0,high,3.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,high,84.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,high,79.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,high,542.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,high,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,high,70.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
for i,cuis in enumerate(df['cuisine']):
    for el in cuis:
        df[el].iloc[i]=1
df

/home/andrei/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,city,cuisine,ranking,rating,price_range,rev_nums,VegetarianFriendly,European,Mediterranean,Italian,...,Ecuadorean,Welsh,Polynesian,Azerbaijani,Fujian,Yunnan,Burmese,Salvadoran,Latvian,Xinjiang
0,Paris,"[European, French, International]",5570.0,3.5,high,194.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Stockholm,"[FastFood, Mediterranean, MiddleEastern, Israe...",1537.0,4.0,high,10.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,mid,688.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin,"[Belgian, European]",3458.0,5.0,high,3.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,high,84.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,high,79.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,high,542.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,high,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,high,70.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
dummies=pd.get_dummies(df['price_range'])
df=pd.concat([df,dummies],axis=1)
dummies=pd.get_dummies(df['city'])
df=pd.concat([df,dummies],axis=1)
df

,city,cuisine,ranking,rating,price_range,rev_nums,VegetarianFriendly,European,Mediterranean,Italian,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,Paris,"[European, French, International]",5570.0,3.5,high,194.0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Stockholm,"[FastFood, Mediterranean, MiddleEastern, Israe...",1537.0,4.0,high,10.0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,London,"[Japanese, Sushi, Asian, Grill, VegetarianFrie...",353.0,4.5,mid,688.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berlin,"[Belgian, European]",3458.0,5.0,high,3.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Munich,"[German, CentralEuropean, VegetarianFriendly]",621.0,4.0,high,84.0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39975,Milan,"[Italian, VegetarianFriendly, VeganOptions, Gl...",500.0,4.5,high,79.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
39976,Paris,"[French, American, Bar, European, VegetarianFr...",6341.0,3.5,high,542.0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
39977,Stockholm,"[Japanese, Sushi]",1652.0,4.5,high,4.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39978,Warsaw,"[Polish, European, EasternEuropean, CentralEur...",641.0,4.0,high,70.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
#X = df.drop(['id', 'rating'], axis = 1)  
X=df.drop(['city','rating','price_range','cuisine'],axis=1)
y = df['rating']  
      
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
      
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

In [14]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
      
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [15]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.21356278139069534
